Given fasta files as specified in file_list, run SyRI_IS to identify the IS positions (SyRI_IS), SVs  (SyRI_IS) and the relationship among ISs of different time-points (classify_IS_events).

# Overview

1. load file_list
2. run SyRI_IS
3. run classify_IS_events against ...
    1. consecutive rounds of sequence
    2. MA ancestor (after FACS enrichment)
    3. MDS42 + IS1
4. Aggeregate stats for other ipynb
    1. Genomesize (03)
    2. Distance of insertions from the closest preexisting IS (04)
    3. position of iss in the gneome (02)
    4. Find identical mutations in different runs to avoid over counting
    5. get statistics of each genome
    6. aggregate sv data (MDS) (02)

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import sys
import os
import glob
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
sys.path.append('../src/SyRI_IS/')
import importlib, subprocess
import run_utils

In [2]:
file_df = pd.read_csv('../fasta/File_list_20240703.csv', na_filter=False)
file_df.head()

,IS_Detect_ID,ParentLine,SubLine,gen,file_name,Anc,RecA,Prefix,sample_name_raw,Contig_Date,Complete,Folder,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
0,,0,0,Anc,20230904/MDS42_IS1.fa,,1,,,,,,,,,
1,L01_Anc,1,1,FACS,20231004/L01_Anc_m1.fasta,R01,1,L01-1,L01_Anc,20220418,TRUE,20231004,,,,1
2,L01_Anc,1,2,FACS,20231004/L01_Anc_m1.fasta,R01,1,L01-2,L01_Anc,20220418,TRUE,20231004,,,,2
3,L01_Anc,1,3,FACS,20231004/L01_Anc_m1.fasta,R01,1,L01-3,L01_Anc,20220418,TRUE,20231004,,,,3
4,L01_Anc,1,4,FACS,20231004/L01_Anc_m1.fasta,R01,1,L01-4,L01_Anc,20220418,TRUE,20231004,,,,4


In [3]:
file_df.tail()

,IS_Detect_ID,ParentLine,SubLine,gen,file_name,Anc,RecA,Prefix,sample_name_raw,Contig_Date,Complete,Folder,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
128,L10-4_G20,10,4,20,20231003/L10-4_G20.fasta,r04,0,L10-4,L10-4_G20,20230909,TRUE,20231003,,,,128
129,L11-1_G20,11,1,20,20231003/L11-1_G20.fasta,r05,0,L11-1,L11-1_G20,20230909,TRUE,20231003,,,,129
130,L11-2_G20,11,2,20,20231003/L11-2_G20.fasta,r05,0,L11-2,L11-2_G20,20230909,TRUE,20231003,,,,130
131,L11-3_G20,11,3,20,20231003/L11-3_G20.fasta,r05,0,L11-3,L11-3_G20,20230909,TRUE,20231003,,,,131
132,L11-4_G20,11,4,20,20231003/L11-4_G20.fasta,r05,0,L11-4,L11-4_G20,20230909,FALSE,20231003,,,,132


In [4]:
master_dir = 'exp/multiple_runs11'
is_detect_dir = 'exp/is_detect11'
tmpdir = os.path.join(master_dir, 'tmp')
exportdir = os.path.join(master_dir, 'export')
logdir = os.path.join(master_dir, 'log')
stderr_out = os.path.join(logdir, 'stderr_out')
os.makedirs(stderr_out, exist_ok=True)
fastadir = '../fasta/'
lines = file_df.Prefix.unique()
lines = [line for line in lines if line != '']
is_fasta = os.path.join(fastadir, 'IS1.fasta')
print(lines[0:5], len(lines))
os.makedirs(logdir, exist_ok=True)

['L01-1', 'L01-2', 'L01-3', 'L01-4', 'L02-1'] 44


# Run SyRI_IS

In [5]:
import subprocess
facs_fasta_files = []
ma_fasta_files = []
gen_ids = []
is_detect_ids = []
#lines_ = [line for line in lines if line not in ['L01-3', 'L01-4', 'L10-1', 'L10-2']]
#lines_ = ['L01-1', 'L01-2']
lines_ = lines
for line in lines_:
	# example python3 ../src/SyRI_IS/runSyRI_IS.py -i ../fasta/IS1.fasta -g ../fasta/MDS42_IS1.fa -g ../fasta/L01_Anc.fasta -g ../fasta/L01-2_G8.fasta -p L01-2 --replicationOrigin 3359187
	mds_fasta = os.path.join(fastadir, file_df.file_name[file_df.gen == 'Anc'][0])
	#parent_line = file_df[file_df.Prefix == line].iloc[0].ParentLine
	C = 'python3 ../src/SyRI_IS/runSyRI_IS.py -i ' + is_fasta 
	C += ' -p ' + line + ' --replicationOrigin 3359187'
	C += ' --tmp ' + tmpdir + ' --out ' + exportdir
	C += ' --log ' + logdir
	C += ' --rerun '
	#C += ' --rerun_align_two_genomes '
	#C += ' --rerun_run_syri '
	C += ' -g ' + mds_fasta
	anc_fasta = mds_fasta
	for gen_id, g in enumerate(file_df[file_df.Prefix == line].file_name):
		fasta_file = os.path.join(fastadir, g)
		print(fasta_file)
		fasta_file = run_utils.get_full_contig(fasta_file, tmpdir=tmpdir, basefasta=anc_fasta, skip_if_exists=False)
		ma_fasta_files.append(fasta_file)
		gen_ids.append(gen_id+2) # exclude the ancstors
		is_detect_ids.append(file_df.query('file_name == @g').iloc[0].IS_Detect_ID)
		C += ' -g ' + fasta_file
		anc_fasta = fasta_file
	print(C)
	# Run the command
	proc = subprocess.Popen(C, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)
	out, err = proc.communicate()

	# Write stdout and stderr to files
	with open(os.path.join(stderr_out, 'std_out_' + line + '.log'), 'w') as f:
		f.write(out.decode('utf-8'))

	with open(os.path.join(stderr_out, 'std_err_' + line + '.log'), 'w') as f:
		f.write(err.decode('utf-8'))

../fasta/20231004/L01_Anc_m1.fasta
contig_cnt 1
../fasta/20231013/L01-1_G8_m.fasta
contig_cnt 1
../fasta/20231003/L01-1_G20.fasta
contig_cnt 1
python3 ../src/SyRI_IS/runSyRI_IS.py -i ../fasta/IS1.fasta -p L01-1 --replicationOrigin 3359187 --tmp exp/multiple_runs11/tmp --out exp/multiple_runs11/export --log exp/multiple_runs11/log --rerun  -g ../fasta/20230904/MDS42_IS1.fa -g ../fasta/20231004/L01_Anc_m1.fasta -g ../fasta/20231013/L01-1_G8_m.fasta -g ../fasta/20231003/L01-1_G20.fasta
../fasta/20231004/L01_Anc_m1.fasta
contig_cnt 1
../fasta/20231003/L01-2_G8.fasta
contig_cnt 1
../fasta/20231003/L01-2_G20.fasta
contig_cnt 1
python3 ../src/SyRI_IS/runSyRI_IS.py -i ../fasta/IS1.fasta -p L01-2 --replicationOrigin 3359187 --tmp exp/multiple_runs11/tmp --out exp/multiple_runs11/export --log exp/multiple_runs11/log --rerun  -g ../fasta/20230904/MDS42_IS1.fa -g ../fasta/20231004/L01_Anc_m1.fasta -g ../fasta/20231003/L01-2_G8.fasta -g ../fasta/20231003/L01-2_G20.fasta
../fasta/20231004/L01_Anc_m1

../fasta/20240703/L05_Anc_m2.fasta
contig_cnt 1
../fasta/20231003/L05-2_G8.fasta
contig_cnt 1
../fasta/20231003/L05-2_G20.fasta
contig_cnt 1
python3 ../src/SyRI_IS/runSyRI_IS.py -i ../fasta/IS1.fasta -p L05-2 --replicationOrigin 3359187 --tmp exp/multiple_runs11/tmp --out exp/multiple_runs11/export --log exp/multiple_runs11/log --rerun  -g ../fasta/20230904/MDS42_IS1.fa -g ../fasta/20240703/L05_Anc_m2.fasta -g ../fasta/20231003/L05-2_G8.fasta -g ../fasta/20231003/L05-2_G20.fasta
../fasta/20240703/L05_Anc_m2.fasta
contig_cnt 1
../fasta/20231003/L05-3_G8.fasta
contig_cnt 1
../fasta/20231003/L05-3_G20.fasta
contig_cnt 1
python3 ../src/SyRI_IS/runSyRI_IS.py -i ../fasta/IS1.fasta -p L05-3 --replicationOrigin 3359187 --tmp exp/multiple_runs11/tmp --out exp/multiple_runs11/export --log exp/multiple_runs11/log --rerun  -g ../fasta/20230904/MDS42_IS1.fa -g ../fasta/20240703/L05_Anc_m2.fasta -g ../fasta/20231003/L05-3_G8.fasta -g ../fasta/20231003/L05-3_G20.fasta
../fasta/20231004/L05_Anc_m1.fas

../fasta/20231004/L09_Anc_m1.fasta
contig_cnt 1
../fasta/20231003/L09-3_G8.fasta
contig_cnt 1
../fasta/20231003/L09-3_G20.fasta
contig_cnt 1
python3 ../src/SyRI_IS/runSyRI_IS.py -i ../fasta/IS1.fasta -p L09-3 --replicationOrigin 3359187 --tmp exp/multiple_runs11/tmp --out exp/multiple_runs11/export --log exp/multiple_runs11/log --rerun  -g ../fasta/20230904/MDS42_IS1.fa -g ../fasta/20231004/L09_Anc_m1.fasta -g ../fasta/20231003/L09-3_G8.fasta -g ../fasta/20231003/L09-3_G20.fasta
../fasta/20231004/L09_Anc_m2.fasta
contig_cnt 1
../fasta/20231003/L09-4_G8.fasta
contig_cnt 1
../fasta/20231003/L09-4_G20.fasta
contig_cnt 1
python3 ../src/SyRI_IS/runSyRI_IS.py -i ../fasta/IS1.fasta -p L09-4 --replicationOrigin 3359187 --tmp exp/multiple_runs11/tmp --out exp/multiple_runs11/export --log exp/multiple_runs11/log --rerun  -g ../fasta/20230904/MDS42_IS1.fa -g ../fasta/20231004/L09_Anc_m2.fasta -g ../fasta/20231003/L09-4_G8.fasta -g ../fasta/20231003/L09-4_G20.fasta
../fasta/20231003/L10_Anc.fasta


In [6]:
if False: # just for updating vectors
	facs_fasta_files = []
	ma_fasta_files = []
	prefixes = []
	gen_ids = []
	is_detect_ids = []
	#lines_ = [line for line in lines if line not in ['L01-3', 'L01-4', 'L10-1', 'L10-2']]
	lines_ = lines
	for line in lines_:
		# example python3 ../src/SyRI_IS/runSyRI_IS.py -i ../fasta/IS1.fasta -g ../fasta/MDS42_IS1.fa -g ../fasta/L01_Anc.fasta -g ../fasta/L01-2_G8.fasta -p L01-2 --replicationOrigin 3359187
		mds_fasta = os.path.join(fastadir, file_df.file_name[file_df.gen == 'Anc'][0])
		#parent_line = file_df[file_df.Prefix == line].iloc[0].ParentLine
		C = 'python3 ../src/SyRI_IS/runSyRI_IS.py -i ' + is_fasta 
		C += ' -p ' + line + ' --replicationOrigin 3359187'
		C += ' --tmp ' + tmpdir + ' --out ' + exportdir
		C += ' --log ' + logdir
		C += ' --rerun '
		C += ' -g ' + mds_fasta
		anc_fasta = mds_fasta
		for gen_id, g in enumerate(file_df[file_df.Prefix == line].file_name):
			fasta_file = os.path.join(fastadir, g)
			fasta_file = run_utils.get_full_contig(fasta_file, tmpdir=tmpdir, basefasta=anc_fasta, skip_if_exists=True)
			ma_fasta_files.append(fasta_file)
			prefixes.append(line)
			gen_ids.append(gen_id+2) # exclude the ancstors
			is_detect_ids.append(file_df.query('file_name == @g').iloc[0].IS_Detect_ID)
			C += ' -g ' + fasta_file
			anc_fasta = fasta_file
		print(C)

## merge plots

In [7]:
# concatenate the runs
merged_out = os.path.join(exportdir, 'merged')
merged_tmp = os.path.join(tmpdir, 'merged')

#lines_ = [line for line in lines if line not in ['L05-1', 'L05-2', 'L05-3', 'L05-4']]
lines_ = lines
C = 'python3 ../src/SyRI_IS/connect_multiple_runs_plotsr.py ' + tmpdir + ' ' + exportdir + ' --tmp ' + merged_tmp + ' --out ' + merged_out
C += ' -p ' + ' -p '.join(lines_)
C += ' -i MDS42_IS1 -i FACS -i MA_G08 -i MA_G20'
C += ' --log ' + os.path.join(logdir, 'merged')
print(C)
!{C}

python3 ../src/SyRI_IS/connect_multiple_runs_plotsr.py exp/multiple_runs11/tmp exp/multiple_runs11/export --tmp exp/multiple_runs11/tmp/merged --out exp/multiple_runs11/export/merged -p L01-1 -p L01-2 -p L01-3 -p L01-4 -p L02-1 -p L02-2 -p L02-3 -p L02-4 -p L03-1 -p L03-2 -p L03-3 -p L03-4 -p L04-1 -p L04-2 -p L04-3 -p L04-4 -p L05-1 -p L05-2 -p L05-3 -p L05-4 -p L06-1 -p L06-2 -p L06-3 -p L06-4 -p L07-1 -p L07-2 -p L07-3 -p L07-4 -p L08-1 -p L08-2 -p L08-3 -p L08-4 -p L09-1 -p L09-2 -p L09-3 -p L09-4 -p L10-1 -p L10-2 -p L10-3 -p L10-4 -p L11-1 -p L11-2 -p L11-3 -p L11-4 -i MDS42_IS1 -i FACS -i MA_G08 -i MA_G20 --log exp/multiple_runs11/log/merged
['--sr', 'exp/multiple_runs11/tmp/merged/syri/merged.0.syri.out', '--sr', 'exp/multiple_runs11/tmp/merged/syri/merged.1.syri.out', '--sr', 'exp/multiple_runs11/tmp/merged/syri/merged.2.syri.out', '--genomes', 'exp/multiple_runs11/tmp/merged/genome_info.txt', '--markers', 'exp/multiple_runs11/tmp/merged/merged_IS_tag.bed', '-s', '100', '-o', 

In [8]:
# ouput svs for supplemntary figure
for parent in ['L01', 'L02', 'L03', 'L04', 'L05', 'L06', 'L07', 'L08', 'L09', 'L10', 'L11']:
	lines__ = [line for line in lines if line.startswith(parent)]
	C = 'python3 ../src/SyRI_IS/connect_multiple_runs_plotsr.py '+ tmpdir + ' ' + exportdir 
	C += ' --tmp ' + os.path.join(merged_tmp, parent) + ' --out ' + os.path.join(merged_out, parent) 
	C += ' -p ' + ' -p '.join(lines__)
	C += ' -i MDS42_IS1 -i FACS -i MA_G08 -i MA_G20'
	C += ' --log ' + os.path.join(logdir, 'merged', parent)
	C += ' --plotsr " -W 3 -H 4 -f 10 --cfg ./20231001_plotsr.cfg --xlim 4490000 --noyaxislabel --noxaxislabel"'
	print(C)
	!{C}

python3 ../src/SyRI_IS/connect_multiple_runs_plotsr.py exp/multiple_runs11/tmp exp/multiple_runs11/export --tmp exp/multiple_runs11/tmp/merged/L01 --out exp/multiple_runs11/export/merged/L01 -p L01-1 -p L01-2 -p L01-3 -p L01-4 -i MDS42_IS1 -i FACS -i MA_G08 -i MA_G20 --log exp/multiple_runs11/log/merged/L01 --plotsr " -W 3 -H 4 -f 10 --cfg ./20231001_plotsr.cfg --xlim 4490000 --noyaxislabel --noxaxislabel"
['--sr', 'exp/multiple_runs11/tmp/merged/L01/syri/merged.0.syri.out', '--sr', 'exp/multiple_runs11/tmp/merged/L01/syri/merged.1.syri.out', '--sr', 'exp/multiple_runs11/tmp/merged/L01/syri/merged.2.syri.out', '--genomes', 'exp/multiple_runs11/tmp/merged/L01/genome_info.txt', '--markers', 'exp/multiple_runs11/tmp/merged/L01/merged_IS_tag.bed', '-s', '100', '-o', 'exp/multiple_runs11/export/merged/L01/merged_plotsr.pdf', '-W', '3', '-H', '4', '-f', '10', '--cfg', './20231001_plotsr.cfg', '--xlim', '4490000', '--noyaxislabel', '--noxaxislabel']
2024-07-03 12:58:53,068 - Plotsr - INFO - S

2024-07-03 12:59:06,138 - Plotsr - INFO - Starting
readbasecfg - ERROR - # is not a valid config parameter. Using default value.
0 -1 4490000 (0.0, 4313004.0)
2024-07-03 12:59:07,632 - Plotsr - INFO - Plot exp/multiple_runs11/export/merged/L04/merged_plotsr.pdf generated.
2024-07-03 12:59:07,632 - Plotsr - INFO - Finished
['--sr', 'exp/multiple_runs11/tmp/merged/L04/syri/merged.0.syri.local.out', '--sr', 'exp/multiple_runs11/tmp/merged/L04/syri/merged.1.syri.local.out', '--sr', 'exp/multiple_runs11/tmp/merged/L04/syri/merged.2.syri.local.out', '--genomes', 'exp/multiple_runs11/tmp/merged/L04/genome_info.txt', '--markers', 'exp/multiple_runs11/tmp/merged/L04/merged_IS_tag.bed', '-s', '100', '-o', 'exp/multiple_runs11/export/merged/L04/merged-local_plotsr.pdf', '-W', '3', '-H', '4', '-f', '10', '--cfg', './20231001_plotsr.cfg', '--xlim', '4490000', '--noyaxislabel', '--noxaxislabel']
2024-07-03 12:59:07,633 - Plotsr - INFO - Starting
2024-07-03 12:59:07,633 - Plotsr - INFO - Starting
rea

0 -1 4490000 (0.0, 3998098.0)
2024-07-03 12:59:21,662 - Plotsr - INFO - Plot exp/multiple_runs11/export/merged/L07/merged-local_plotsr.pdf generated.
2024-07-03 12:59:21,662 - Plotsr - INFO - Plot exp/multiple_runs11/export/merged/L07/merged-local_plotsr.pdf generated.
2024-07-03 12:59:21,662 - Plotsr - INFO - Finished
2024-07-03 12:59:21,662 - Plotsr - INFO - Finished
python3 ../src/SyRI_IS/connect_multiple_runs_plotsr.py exp/multiple_runs11/tmp exp/multiple_runs11/export --tmp exp/multiple_runs11/tmp/merged/L08 --out exp/multiple_runs11/export/merged/L08 -p L08-1 -p L08-2 -p L08-3 -p L08-4 -i MDS42_IS1 -i FACS -i MA_G08 -i MA_G20 --log exp/multiple_runs11/log/merged/L08 --plotsr " -W 3 -H 4 -f 10 --cfg ./20231001_plotsr.cfg --xlim 4490000 --noyaxislabel --noxaxislabel"
['--sr', 'exp/multiple_runs11/tmp/merged/L08/syri/merged.0.syri.out', '--sr', 'exp/multiple_runs11/tmp/merged/L08/syri/merged.1.syri.out', '--sr', 'exp/multiple_runs11/tmp/merged/L08/syri/merged.2.syri.out', '--genomes

['--sr', 'exp/multiple_runs11/tmp/merged/L11/syri/merged.0.syri.out', '--sr', 'exp/multiple_runs11/tmp/merged/L11/syri/merged.1.syri.out', '--sr', 'exp/multiple_runs11/tmp/merged/L11/syri/merged.2.syri.out', '--genomes', 'exp/multiple_runs11/tmp/merged/L11/genome_info.txt', '--markers', 'exp/multiple_runs11/tmp/merged/L11/merged_IS_tag.bed', '-s', '100', '-o', 'exp/multiple_runs11/export/merged/L11/merged_plotsr.pdf', '-W', '3', '-H', '4', '-f', '10', '--cfg', './20231001_plotsr.cfg', '--xlim', '4490000', '--noyaxislabel', '--noxaxislabel']
2024-07-03 12:59:35,926 - Plotsr - INFO - Starting
readbasecfg - ERROR - # is not a valid config parameter. Using default value.
0 -1 4490000 (0.0, 4248613.0)
2024-07-03 12:59:37,418 - Plotsr - INFO - Plot exp/multiple_runs11/export/merged/L11/merged_plotsr.pdf generated.
2024-07-03 12:59:37,418 - Plotsr - INFO - Finished
['--sr', 'exp/multiple_runs11/tmp/merged/L11/syri/merged.0.syri.local.out', '--sr', 'exp/multiple_runs11/tmp/merged/L11/syri/merg

In [9]:
# Combine every four pdfs to one page
merged_out = os.path.join(exportdir, 'merged')
merged_tmp = os.path.join(tmpdir, 'merged')
C = 'pdfunite '
for parent in ['L01', 'L02', 'L03', 'L04', 'L05', 'L06', 'L07', 'L08', 'L09', 'L10', 'L11']:
	C += os.path.join(merged_out, parent, 'merged-local_plotsr.pdf') + ' '
C += ' ./combine_pdf/combined_plotsr.pdf'
print(C)
!{C}
!{'cd ./combine_pdf && pdflatex latex_script_to_combine_plotsr_pdfs.tex'}

pdfunite exp/multiple_runs11/export/merged/L01/merged-local_plotsr.pdf exp/multiple_runs11/export/merged/L02/merged-local_plotsr.pdf exp/multiple_runs11/export/merged/L03/merged-local_plotsr.pdf exp/multiple_runs11/export/merged/L04/merged-local_plotsr.pdf exp/multiple_runs11/export/merged/L05/merged-local_plotsr.pdf exp/multiple_runs11/export/merged/L06/merged-local_plotsr.pdf exp/multiple_runs11/export/merged/L07/merged-local_plotsr.pdf exp/multiple_runs11/export/merged/L08/merged-local_plotsr.pdf exp/multiple_runs11/export/merged/L09/merged-local_plotsr.pdf exp/multiple_runs11/export/merged/L10/merged-local_plotsr.pdf exp/multiple_runs11/export/merged/L11/merged-local_plotsr.pdf  ./combine_pdf/combined_plotsr.pdf
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
(./latex_script_to_combine_plotsr_pdfs.tex
LaTeX2e <2017-04-15>
Babel <3.18> and hyphenation patterns for 3 language(s) loa

# Classify the events
Determine dup, del. Connect ISs of different generations. Determine simple insertions etc.

In [10]:
lines_ = lines
for i, line in enumerate(lines_):
	C_ = 'python3 ../src/SyRI_IS/classify_IS_events.py '
	C_ += ' --prefix ' + line 
	C_ += ' --out ' + os.path.join(exportdir, 'classify_IS_events')
	C_ += ' --log ' + logdir
	C_ += ' --input ' + os.path.join(tmpdir, line)
	C_ += ' --calc_depth_interval '
	for gen_id, g in enumerate(file_df[file_df.Prefix == line].file_name):
		if gen_id == 0: # 0: FACS, 1: MA_G08, 2: MA_G20
			continue
		C = C_ + ' --rid ' + str(gen_id+0) + ' --qid ' + str(gen_id+1)
		if i ==0 and gen_id == 1:
			print(C)
		proc = subprocess.Popen(C, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)
		out, err = proc.communicate()

		# Write stdout and stderr to files
		with open(os.path.join(stderr_out, 'std_out_' + line + '.log'), 'w') as f:
			f.write(out.decode('utf-8'))

		with open(os.path.join(stderr_out, 'std_err_' + line + '.log'), 'w') as f:
			f.write(err.decode('utf-8'))

python3 ../src/SyRI_IS/classify_IS_events.py  --prefix L01-1 --out exp/multiple_runs11/export/classify_IS_events --log exp/multiple_runs11/log --input exp/multiple_runs11/tmp/L01-1 --calc_depth_interval  --rid 1 --qid 2


In [11]:
depth_interval_df_evo = pd.DataFrame()
lines_ = lines
for line in lines_:
	for qid in [2,3]:
		df_ = pd.read_csv(os.path.join(exportdir, 'classify_IS_events', line, f'{line}_genome.{qid}.depth_interval.csv'))
		df_['line'] = line
		df_['gen_id'] = qid
		depth_interval_df_evo = pd.concat([depth_interval_df_evo, df_], axis=0)
depth_interval_df_evo= depth_interval_df_evo.reset_index(drop=True)
depth_interval_df_evo['gen_id'] = depth_interval_df_evo['gen_id'].astype(int)
depth_interval_df_evo.to_csv(os.path.join(exportdir, 'classify_IS_events', 'depth_interval.csv'), index=False)
depth_interval_df_evo.head()

,group,start,end,depth,length,start_fix,end_fix,is_start,is_end,line,gen_id
0,1,1,549437,1.0,549437,1,552530,-1,3,L01-1,2
1,2,549438,554384,0.0,4947,552531,560570,3,6,L01-1,2
2,3,554385,2633390,1.0,2079006,560571,2661330,6,21,L01-1,2
3,4,2633391,2637598,0.0,4208,2661331,2665538,21,22,L01-1,2
4,5,2637599,3965065,1.0,1327467,2665539,4003851,22,-1,L01-1,2


## IS events compared to FACS generation
The previous analysis was based on the comparison between two consecutive generations. Here we compare the IS events to the FACS generation.
currently, simple transposition is ignored

In [12]:
import is_event_classification_func as iecf
max_run = 1
run_cnt =0
for i, line in enumerate(lines_):
	C_ = 'python3 ../src/SyRI_IS/classify_IS_events.py '
	C_ += ' --prefix ' + line 
	C_ += ' --out ' + os.path.join(exportdir, 'classify_IS_events_Anc')
	C_ += ' --log ' + logdir
	C_ += ' --input ' + os.path.join(tmpdir, line)
	C_ += ' --calc_depth_interval ' 
	for gen_id, g in enumerate(file_df[file_df.Prefix == line].file_name):
		# genid 0 is the ancestor obtained from FACS
		if gen_id == 0:
			continue
		C = C_ + ' --rid ' + str(1) + ' --qid ' + str(gen_id+1)
		if i ==0 and gen_id == 0:
			print(C)
		iecf.run_syri_isrm_for_specific_id(tmpdir, line, 1, gen_id+1)
		os.system(C)

[M::mm_idx_gen::0.057*1.01] collected minimizers
[M::mm_idx_gen::0.076*1.00] sorted minimizers
[M::main::0.076*1.00] loaded/built the index for 1 target sequence(s)
[M::mm_mapopt_update::0.076*1.00] mid_occ = 100
[M::mm_idx_stat] kmer size: 19; skip: 10; is_hpc: 1; #seq: 1
[M::mm_idx_stat::0.080*1.00] distinct minimizers: 523880 (99.33% are singletons); average occurrences: 1.015; average spacing: 7.460; total length: 3965065
[M::worker_pipeline::0.620*0.99] mapped 1 sequences
[M::main] Version: 2.19-r1057
[M::main] CMD: minimap2 -ax asm20 --eqx -H -f 100 -r1k,10k --rmq=no -t 1 exp/multiple_runs11/tmp/L01-1/is_deleted/L01-1_genome.deleted_IS.1.fasta exp/multiple_runs11/tmp/L01-1/is_deleted/L01-1_genome.deleted_IS.2.fasta
[M::main] Real time: 0.623 sec; CPU: 0.616 sec; Peak RSS: 0.087 GB
local_variation - WARNING - Finished syri


TLOut.txt  is empty. Skipping analysing it.
invTLOut.txt  is empty. Skipping analysing it.
ctxOut.txt is empty. Skipping analysing it.
TL Out.txt is empty. Skipping analysing it.
invTL Out.txt is empty. Skipping analysing it.
dup Out.txt is empty. Skipping analysing it.
invDup Out.txt is empty. Skipping analysing it.
ctxOut.txt is empty. Skipping analysing it.
TLOut.txt  is empty. Skipping analysing it.
invTLOut.txt  is empty. Skipping analysing it.
ctxOut.txt is empty. Skipping analysing it.


[M::mm_idx_gen::0.057*1.01] collected minimizers
[M::mm_idx_gen::0.076*1.00] sorted minimizers
[M::main::0.076*1.00] loaded/built the index for 1 target sequence(s)
[M::mm_mapopt_update::0.076*1.00] mid_occ = 100
[M::mm_idx_stat] kmer size: 19; skip: 10; is_hpc: 1; #seq: 1
[M::mm_idx_stat::0.080*1.00] distinct minimizers: 523880 (99.33% are singletons); average occurrences: 1.015; average spacing: 7.460; total length: 3965065
[M::worker_pipeline::0.605*0.99] mapped 1 sequences
[M::main] Version: 2.19-r1057
[M::main] CMD: minimap2 -ax asm20 --eqx -H -f 100 -r1k,10k --rmq=no -t 1 exp/multiple_runs11/tmp/L01-1/is_deleted/L01-1_genome.deleted_IS.1.fasta exp/multiple_runs11/tmp/L01-1/is_deleted/L01-1_genome.deleted_IS.3.fasta
[M::main] Real time: 0.608 sec; CPU: 0.601 sec; Peak RSS: 0.087 GB
local_variation - WARNING - Finished syri


invTLOut.txt  is empty. Skipping analysing it.
ctxOut.txt is empty. Skipping analysing it.
invTL Out.txt is empty. Skipping analysing it.
dup Out.txt is empty. Skipping analysing it.
invDup Out.txt is empty. Skipping analysing it.
ctxOut.txt is empty. Skipping analysing it.
invTLOut.txt  is empty. Skipping analysing it.
ctxOut.txt is empty. Skipping analysing it.


[M::mm_idx_gen::0.058*1.01] collected minimizers
[M::mm_idx_gen::0.077*1.00] sorted minimizers
[M::main::0.077*1.00] loaded/built the index for 1 target sequence(s)
[M::mm_mapopt_update::0.077*1.00] mid_occ = 100
[M::mm_idx_stat] kmer size: 19; skip: 10; is_hpc: 1; #seq: 1
[M::mm_idx_stat::0.081*1.00] distinct minimizers: 523880 (99.33% are singletons); average occurrences: 1.015; average spacing: 7.460; total length: 3965065
[M::worker_pipeline::0.621*0.99] mapped 1 sequences
[M::main] Version: 2.19-r1057
[M::main] CMD: minimap2 -ax asm20 --eqx -H -f 100 -r1k,10k --rmq=no -t 1 exp/multiple_runs11/tmp/L01-2/is_deleted/L01-2_genome.deleted_IS.1.fasta exp/multiple_runs11/tmp/L01-2/is_deleted/L01-2_genome.deleted_IS.2.fasta
[M::main] Real time: 0.624 sec; CPU: 0.617 sec; Peak RSS: 0.087 GB
local_variation - WARNING - Finished syri


...

invOut.txt  is empty. Skipping analysing it.
TLOut.txt  is empty. Skipping analysing it.
invTLOut.txt  is empty. Skipping analysing it.
ctxOut.txt is empty. Skipping analysing it.
inv Out.txt is empty. Skipping analysing it.
TL Out.txt is empty. Skipping analysing it.
invTL Out.txt is empty. Skipping analysing it.
dup Out.txt is empty. Skipping analysing it.
invDup Out.txt is empty. Skipping analysing it.
ctxOut.txt is empty. Skipping analysing it.
invOut.txt  is empty. Skipping analysing it.
TLOut.txt  is empty. Skipping analysing it.
invTLOut.txt  is empty. Skipping analysing it.
ctxOut.txt is empty. Skipping analysing it.


[M::mm_idx_gen::0.057*1.01] collected minimizers
[M::mm_idx_gen::0.076*1.00] sorted minimizers
[M::main::0.076*1.00] loaded/built the index for 1 target sequence(s)
[M::mm_mapopt_update::0.076*1.00] mid_occ = 100
[M::mm_idx_stat] kmer size: 19; skip: 10; is_hpc: 1; #seq: 1
[M::mm_idx_stat::0.079*1.00] distinct minimizers: 524338 (99.34% are singletons); average occurrences: 1.014; average spacing: 7.460; total length: 3967244
[M::worker_pipeline::0.620*0.99] mapped 1 sequences
[M::main] Version: 2.19-r1057
[M::main] CMD: minimap2 -ax asm20 --eqx -H -f 100 -r1k,10k --rmq=no -t 1 exp/multiple_runs11/tmp/L11-4/is_deleted/L11-4_genome.deleted_IS.1.fasta exp/multiple_runs11/tmp/L11-4/is_deleted/L11-4_genome.deleted_IS.2.fasta
[M::main] Real time: 0.623 sec; CPU: 0.617 sec; Peak RSS: 0.087 GB
local_variation - WARNING - Finished syri


TLOut.txt  is empty. Skipping analysing it.
invTLOut.txt  is empty. Skipping analysing it.
ctxOut.txt is empty. Skipping analysing it.
TL Out.txt is empty. Skipping analysing it.
invTL Out.txt is empty. Skipping analysing it.
dup Out.txt is empty. Skipping analysing it.
invDup Out.txt is empty. Skipping analysing it.
ctxOut.txt is empty. Skipping analysing it.
TLOut.txt  is empty. Skipping analysing it.
invTLOut.txt  is empty. Skipping analysing it.
ctxOut.txt is empty. Skipping analysing it.


[M::mm_idx_gen::0.056*1.01] collected minimizers
[M::mm_idx_gen::0.075*1.00] sorted minimizers
[M::main::0.075*1.00] loaded/built the index for 1 target sequence(s)
[M::mm_mapopt_update::0.075*1.00] mid_occ = 100
[M::mm_idx_stat] kmer size: 19; skip: 10; is_hpc: 1; #seq: 1
[M::mm_idx_stat::0.079*1.00] distinct minimizers: 524338 (99.34% are singletons); average occurrences: 1.014; average spacing: 7.460; total length: 3967244
[M::worker_pipeline::0.654*0.99] mapped 1 sequences
[M::main] Version: 2.19-r1057
[M::main] CMD: minimap2 -ax asm20 --eqx -H -f 100 -r1k,10k --rmq=no -t 1 exp/multiple_runs11/tmp/L11-4/is_deleted/L11-4_genome.deleted_IS.1.fasta exp/multiple_runs11/tmp/L11-4/is_deleted/L11-4_genome.deleted_IS.3.fasta
[M::main] Real time: 0.657 sec; CPU: 0.650 sec; Peak RSS: 0.090 GB
local_variation - WARNING - Finished syri


TLOut.txt  is empty. Skipping analysing it.
invTLOut.txt  is empty. Skipping analysing it.
ctxOut.txt is empty. Skipping analysing it.
TL Out.txt is empty. Skipping analysing it.
invTL Out.txt is empty. Skipping analysing it.
dup Out.txt is empty. Skipping analysing it.
ctxOut.txt is empty. Skipping analysing it.
TLOut.txt  is empty. Skipping analysing it.
invTLOut.txt  is empty. Skipping analysing it.
ctxOut.txt is empty. Skipping analysing it.


In [13]:
lines_ = lines
depth_interval_df_anc = pd.DataFrame()
for line in lines_:
	for qid in [2,3]:
		df_ = pd.read_csv(os.path.join(exportdir, 'classify_IS_events_Anc', line, f'{line}_genome.{qid}.depth_interval.csv'))
		df_['line'] = line
		df_['gen_id'] = qid
		depth_interval_df_anc = pd.concat([depth_interval_df_anc, df_], axis=0)
depth_interval_df_anc= depth_interval_df_anc.reset_index(drop=True)
depth_interval_df_anc['gen_id'] = depth_interval_df_anc['gen_id'].astype(int)
depth_interval_df_anc.to_csv(os.path.join(exportdir, 'classify_IS_events_Anc', 'depth_interval.asm20.csv'), index=False)

## IS events compared to MDS42
The previous analysis was based on the comparison between two consecutive generations. Here we compare the IS events to the MDS42 generation.

In [14]:
# I decided to include data for comparison of FACS generation as a negative control of IS mediated deletion
import is_event_classification_func as iecf
#lines_ = ['L05-1', 'L05-2', 'L05-3', 'L05-4']
lines_ = lines
for i, line in enumerate(lines_):
	C_ = 'python3 ../src/SyRI_IS/classify_IS_events.py '
	C_ += ' --prefix ' + line 
	C_ += ' --out ' + os.path.join(exportdir, 'classify_IS_events_MDS')
	C_ += ' --log ' + logdir
	C_ += ' --input ' + os.path.join(tmpdir, line)
	C_ += ' --calc_depth_interval '
	for gen_id, g in enumerate(file_df[file_df.Prefix == line].file_name):
		C = C_ + ' --rid ' + str(0) + ' --qid ' + str(gen_id+1)
		if i ==0 and gen_id == 0:
			print(C)
		iecf.run_syri_isrm_for_specific_id(tmpdir, line, 0, gen_id+1)
		os.system(C)

python3 ../src/SyRI_IS/classify_IS_events.py  --prefix L01-1 --out exp/multiple_runs11/export/classify_IS_events_MDS --log exp/multiple_runs11/log --input exp/multiple_runs11/tmp/L01-1 --calc_depth_interval  --rid 0 --qid 1


[M::mm_idx_gen::0.058*1.01] collected minimizers
[M::mm_idx_gen::0.076*1.00] sorted minimizers
[M::main::0.076*1.00] loaded/built the index for 1 target sequence(s)
[M::mm_mapopt_update::0.076*1.00] mid_occ = 100
[M::mm_idx_stat] kmer size: 19; skip: 10; is_hpc: 1; #seq: 1
[M::mm_idx_stat::0.080*1.00] distinct minimizers: 525478 (99.34% are singletons); average occurrences: 1.015; average spacing: 7.461; total length: 3977333
[M::worker_pipeline::0.611*0.99] mapped 1 sequences
[M::main] Version: 2.19-r1057
[M::main] CMD: minimap2 -ax asm20 --eqx -H -f 100 -r1k,10k --rmq=no -t 1 exp/multiple_runs11/tmp/L01-1/is_deleted/L01-1_genome.deleted_IS.0.fasta exp/multiple_runs11/tmp/L01-1/is_deleted/L01-1_genome.deleted_IS.1.fasta
[M::main] Real time: 0.614 sec; CPU: 0.608 sec; Peak RSS: 0.087 GB
local_variation - WARNING - Finished syri


TLOut.txt  is empty. Skipping analysing it.
invTLOut.txt  is empty. Skipping analysing it.
ctxOut.txt is empty. Skipping analysing it.
TL Out.txt is empty. Skipping analysing it.
invTL Out.txt is empty. Skipping analysing it.
dup Out.txt is empty. Skipping analysing it.
invDup Out.txt is empty. Skipping analysing it.
ctxOut.txt is empty. Skipping analysing it.
TLOut.txt  is empty. Skipping analysing it.
invTLOut.txt  is empty. Skipping analysing it.
ctxOut.txt is empty. Skipping analysing it.


[M::mm_idx_gen::0.057*1.01] collected minimizers
[M::mm_idx_gen::0.075*1.00] sorted minimizers
[M::main::0.075*1.00] loaded/built the index for 1 target sequence(s)
[M::mm_mapopt_update::0.075*1.00] mid_occ = 100
[M::mm_idx_stat] kmer size: 19; skip: 10; is_hpc: 1; #seq: 1
[M::mm_idx_stat::0.079*1.00] distinct minimizers: 525478 (99.34% are singletons); average occurrences: 1.015; average spacing: 7.461; total length: 3977333
[M::worker_pipeline::0.611*0.99] mapped 1 sequences
[M::main] Version: 2.19-r1057
[M::main] CMD: minimap2 -ax asm20 --eqx -H -f 100 -r1k,10k --rmq=no -t 1 exp/multiple_runs11/tmp/L01-1/is_deleted/L01-1_genome.deleted_IS.0.fasta exp/multiple_runs11/tmp/L01-1/is_deleted/L01-1_genome.deleted_IS.2.fasta
[M::main] Real time: 0.613 sec; CPU: 0.607 sec; Peak RSS: 0.090 GB
local_variation - WARNING - Finished syri


invTLOut.txt  is empty. Skipping analysing it.
ctxOut.txt is empty. Skipping analysing it.
invTL Out.txt is empty. Skipping analysing it.
dup Out.txt is empty. Skipping analysing it.
invDup Out.txt is empty. Skipping analysing it.
ctxOut.txt is empty. Skipping analysing it.
invTLOut.txt  is empty. Skipping analysing it.
ctxOut.txt is empty. Skipping analysing it.


...

invOut.txt  is empty. Skipping analysing it.
TLOut.txt  is empty. Skipping analysing it.
invTLOut.txt  is empty. Skipping analysing it.
ctxOut.txt is empty. Skipping analysing it.
inv Out.txt is empty. Skipping analysing it.
TL Out.txt is empty. Skipping analysing it.
invTL Out.txt is empty. Skipping analysing it.
dup Out.txt is empty. Skipping analysing it.
invDup Out.txt is empty. Skipping analysing it.
ctxOut.txt is empty. Skipping analysing it.
invOut.txt  is empty. Skipping analysing it.
TLOut.txt  is empty. Skipping analysing it.
invTLOut.txt  is empty. Skipping analysing it.
ctxOut.txt is empty. Skipping analysing it.


[M::mm_idx_gen::0.058*1.01] collected minimizers
[M::mm_idx_gen::0.076*1.00] sorted minimizers
[M::main::0.076*1.00] loaded/built the index for 1 target sequence(s)
[M::mm_mapopt_update::0.076*1.00] mid_occ = 100
[M::mm_idx_stat] kmer size: 19; skip: 10; is_hpc: 1; #seq: 1
[M::mm_idx_stat::0.080*1.00] distinct minimizers: 525478 (99.34% are singletons); average occurrences: 1.015; average spacing: 7.461; total length: 3977333
[M::worker_pipeline::0.673*0.99] mapped 1 sequences
[M::main] Version: 2.19-r1057
[M::main] CMD: minimap2 -ax asm20 --eqx -H -f 100 -r1k,10k --rmq=no -t 1 exp/multiple_runs11/tmp/L11-4/is_deleted/L11-4_genome.deleted_IS.0.fasta exp/multiple_runs11/tmp/L11-4/is_deleted/L11-4_genome.deleted_IS.2.fasta
[M::main] Real time: 0.675 sec; CPU: 0.669 sec; Peak RSS: 0.170 GB
local_variation - WARNING - Finished syri


TLOut.txt  is empty. Skipping analysing it.
invTLOut.txt  is empty. Skipping analysing it.
ctxOut.txt is empty. Skipping analysing it.
TL Out.txt is empty. Skipping analysing it.
invTL Out.txt is empty. Skipping analysing it.
dup Out.txt is empty. Skipping analysing it.
invDup Out.txt is empty. Skipping analysing it.
ctxOut.txt is empty. Skipping analysing it.
TLOut.txt  is empty. Skipping analysing it.
invTLOut.txt  is empty. Skipping analysing it.
ctxOut.txt is empty. Skipping analysing it.


[M::mm_idx_gen::0.058*1.01] collected minimizers
[M::mm_idx_gen::0.077*1.00] sorted minimizers
[M::main::0.077*1.00] loaded/built the index for 1 target sequence(s)
[M::mm_mapopt_update::0.077*1.00] mid_occ = 100
[M::mm_idx_stat] kmer size: 19; skip: 10; is_hpc: 1; #seq: 1
[M::mm_idx_stat::0.081*1.00] distinct minimizers: 525478 (99.34% are singletons); average occurrences: 1.015; average spacing: 7.461; total length: 3977333
[M::worker_pipeline::0.707*0.99] mapped 1 sequences
[M::main] Version: 2.19-r1057
[M::main] CMD: minimap2 -ax asm20 --eqx -H -f 100 -r1k,10k --rmq=no -t 1 exp/multiple_runs11/tmp/L11-4/is_deleted/L11-4_genome.deleted_IS.0.fasta exp/multiple_runs11/tmp/L11-4/is_deleted/L11-4_genome.deleted_IS.3.fasta
[M::main] Real time: 0.710 sec; CPU: 0.703 sec; Peak RSS: 0.172 GB
local_variation - WARNING - Finished syri


TLOut.txt  is empty. Skipping analysing it.
invTLOut.txt  is empty. Skipping analysing it.
ctxOut.txt is empty. Skipping analysing it.
TL Out.txt is empty. Skipping analysing it.
invTL Out.txt is empty. Skipping analysing it.
dup Out.txt is empty. Skipping analysing it.
ctxOut.txt is empty. Skipping analysing it.
TLOut.txt  is empty. Skipping analysing it.
invTLOut.txt  is empty. Skipping analysing it.
ctxOut.txt is empty. Skipping analysing it.


In [15]:
depth_interval_df = pd.DataFrame()
for line in lines_:
	for qid in [1, 2,3]:
		df_ = pd.read_csv(os.path.join(exportdir, 'classify_IS_events_MDS', line, f'{line}_genome.{qid}.depth_interval.csv'))
		df_['line'] = line
		df_['gen_id'] = qid
		depth_interval_df = pd.concat([depth_interval_df, df_], axis=0)
depth_interval_df= depth_interval_df.reset_index(drop=True)
depth_interval_df['gen_id'] = depth_interval_df['gen_id'].astype(int)
depth_interval_df.to_csv(os.path.join(exportdir, 'classify_IS_events_MDS', 'depth_interval.asm20.csv'), index=False)

# Obtain data for analysis
### Genomesize

In [16]:
import SyRI_IS
#genome_size_df = pd.DataFrame({'Prefix': prefixes, 'Gen': gen_ids, 'File': ma_fasta_files})
lines_ = lines
genome_size_df = pd.DataFrame({'Prefix': [line for line in lines_ for _ in range(3)], 'Gen': gen_ids, 'File': ma_fasta_files})
genome_size_df['Gen'] = genome_size_df.Gen.apply(lambda x: x-1)
genome_size_df['genome_size'] = genome_size_df.File.apply(lambda x: SyRI_IS.get_fasta_length(x, rmN=True))
genome_size_df.head()

,Prefix,Gen,File,genome_size
0,L01-1,1,../fasta/20231004/L01_Anc_m1.fasta,4003851
1,L01-1,2,../fasta/20231013/L01-1_G8_m.fasta,4014776
2,L01-1,3,../fasta/20231003/L01-1_G20.fasta,3923637
3,L01-2,1,../fasta/20231004/L01_Anc_m1.fasta,4003851
4,L01-2,2,../fasta/20231003/L01-2_G8.fasta,4007666


### insertion distance

In [17]:
import analyse_stats
insertion_distance_file = analyse_stats.get_insertion_distances(lines_, exportdir, [2, 3])
print(insertion_distance_file.shape)
insertion_distance_file.to_csv(os.path.join(exportdir, 'classify_IS_events', 'simple_insertion_distances.csv'), index=False)
insertion_distance_file.tail(10)

(904, 8)


,insert_id,pos,position_status,sv,closest_original_id,closest_original_distance,Line,Gen
10,31,2953083,new,simple_insertion,30,54630,L11-4,2
11,32,3390500,new,simple_insertion,33,271391,L11-4,2
12,37,3980006,new,simple_insertion,36,57312,L11-4,2
0,13,1244239,new,simple_insertion,14,45894,L11-4,3
1,18,1576177,new,simple_insertion,17,136310,L11-4,3
2,21,1873190,new,unknown,22,188,L11-4,3
3,26,2072935,new,simple_insertion,27,54595,L11-4,3
4,31,2365293,new,simple_insertion,32,80918,L11-4,3
5,46,3757398,new,unknown,45,91996,L11-4,3
6,49,3978936,new,unknown,50,1470,L11-4,3


### position of iss in the gneome

In [18]:
import importlib
importlib.reload(analyse_stats)

<module 'analyse_stats' from '/home/kanai/documents/analysis/myTELR/syri/ipynb/../src/SyRI_IS/analyse_stats.py'>

In [19]:
import analyse_stats
is_positions = analyse_stats.get_is_positions(lines_, tmpdir, [1, 2, 3])
print(is_positions.shape)
is_positions.to_csv(os.path.join(exportdir, 'classify_IS_events', 'IS_positions.csv'), index=False)
is_positions.tail(10)

(2369, 8)


,start,end,IS_strand,max_alignment_length,length,cluster_id,Line,Gen
19,2829024,2832116,forward,3094,3093,19,L11-4,3
20,2902961,2906053,forward,3093,3093,20,L11-4,3
21,2960630,2963721,reverse,3093,3092,21,L11-4,3
22,3401008,3404097,forward,3093,3090,22,L11-4,3
23,3675429,3678520,forward,3093,3092,23,L11-4,3
24,3770500,3773590,forward,3093,3091,24,L11-4,3
25,3854577,3857669,reverse,3093,3093,25,L11-4,3
26,3996588,3999678,forward,3092,3091,26,L11-4,3
27,4162242,4165333,forward,3092,3092,27,L11-4,3
28,4221165,4224256,forward,3093,3092,28,L11-4,3


## Find identical mutations in different runs to avoid over counting

Due to high mutation rate, there seems to be some mutations happening after sequencing directly after IS enrichment, and passage by MA. Here, I detect those events by identifying simlilar IS insertions in predecessors. This data and data exported by plotsr are used to check similar events to avoid overcounting.
1. get all the insertions in terms of the FACS generation
2. Cluster the positions
3. list up all the mutations shared among multiple samples
4. fix the ancestor sequences by hand

In [20]:
import analyse_stats
importlib.reload(analyse_stats)

<module 'analyse_stats' from '/home/kanai/documents/analysis/myTELR/syri/ipynb/../src/SyRI_IS/analyse_stats.py'>

In [21]:
lines_ = lines
is_in_r_df = analyse_stats.get_is_positions_in_ref_genome(lines_, os.path.join(exportdir, 'classify_IS_events'), [1, 2, 3])
print(is_in_r_df.shape)
is_in_r_df.to_csv(os.path.join(exportdir, 'classify_IS_events', 'IS_positions_in_ref_genome.csv'), index=False)
is_in_r_df.tail(5)

(6357, 12)


,pos_id,IS_strand,flank,pos,genome,cluster_id,insert_id,flankMatchDir,position_status,sv,Line,Gen
6352,26,True,r,3757392,Query,6,46,False,new,unknown,L11-4,3
6353,27,True,f,3919996,Query,4,47,True,original,pristine,L11-4,3
6354,27,True,r,3923084,Query,48,48,False,original,pristine,L11-4,3
6355,28,True,f,3978936,Query,49,49,True,new,unknown,L11-4,3
6356,28,True,r,3983498,Query,23,51,False,original,unknown,L11-4,3


In [22]:
is_in_r_df_identical = analyse_stats.get_identical_inserts(is_in_r_df, ['L01', 'L02', 'L03', 'L04', 'L05', 'L06', 'L07', 'L08', 'L09', 'L10', 'L11'])
print(is_in_r_df_identical[0].shape)
is_in_r_df_identical[0].to_csv(os.path.join(exportdir, 'classify_IS_events', 'IS_positions_in_ref_genome_identical.csv'), index=False)
is_in_r_df_identical[0].head()

(2, 15)


,pos_id,IS_strand,flank,pos,genome,cluster_id,insert_id,flankMatchDir,position_status,sv,Line,Gen,pos_cluster,cluster_id_cluster,insert_id_cluster
32,1,False,r,1232077,Query,34,2,True,new,unknown,L02-1,2,1232077,0,11
187,2,False,f,1232095,Query,24,4,False,new,unknown,L02-3,2,1232095,0,11


If similar insertions are detected, I manually checked if they seem to be identical insertions, incorporated into the fasta files and reran the analysis. L02-1 and L02-3 had close insertions but the insertions were in slightly different locus that was the same in the 20th passage so they were considered independent insertions.

## Get statistics of each genome

Input specification of lines
Output Insertion, IS-related events, loss, Copy number change

In [23]:
genome_stats = file_df.drop(file_df.columns[11:], axis =1).drop(file_df.index[0], axis=0)
gen_to_gen_id = pd.DataFrame({'gen': ['Anc', 'FACS', '8', '20'],
	'gen_id': [0, 1, 2, 3]})
genome_stats = genome_stats.merge(gen_to_gen_id, on='gen')
genome_stats = genome_stats.merge(genome_size_df.rename(columns={'Gen': 'gen_id'}),
	on=['Prefix', 'gen_id'])
genome_stats.head()

,IS_Detect_ID,ParentLine,SubLine,gen,file_name,Anc,RecA,Prefix,sample_name_raw,Contig_Date,Complete,gen_id,File,genome_size
0,L01_Anc,1,1,FACS,20231004/L01_Anc_m1.fasta,R01,1,L01-1,L01_Anc,20220418,TRUE,1,../fasta/20231004/L01_Anc_m1.fasta,4003851
1,L01_Anc,1,2,FACS,20231004/L01_Anc_m1.fasta,R01,1,L01-2,L01_Anc,20220418,TRUE,1,../fasta/20231004/L01_Anc_m1.fasta,4003851
2,L01_Anc,1,3,FACS,20231004/L01_Anc_m1.fasta,R01,1,L01-3,L01_Anc,20220418,TRUE,1,../fasta/20231004/L01_Anc_m1.fasta,4003851
3,L01_Anc,1,4,FACS,20231004/L01_Anc_m1.fasta,R01,1,L01-4,L01_Anc,20220418,TRUE,1,../fasta/20231004/L01_Anc_m1.fasta,4003851
4,L02_Anc,2,1,FACS,20231003/L02_Anc.fasta,R02,1,L02-1,L02_Anc,20220418,TRUE,1,../fasta/20231003/L02_Anc.fasta,4026846


In [24]:
genome_stats = pd.merge(genome_stats, is_positions.groupby(['Line', 'Gen']).\
	agg(is_cnt = ('length', 'count'), is_length = ('length', 'sum')).reset_index(),
 	how='left', left_on=['Prefix', 'gen_id'], right_on=['Line', 'Gen']).\
				 	drop(['Line', 'Gen'], axis=1)
genome_stats.tail(2)

,IS_Detect_ID,ParentLine,SubLine,gen,file_name,Anc,RecA,Prefix,sample_name_raw,Contig_Date,Complete,gen_id,File,genome_size,is_cnt,is_length
130,L11-3_G20,11,3,20,20231003/L11-3_G20.fasta,r05,0,L11-3,L11-3_G20,20230909,TRUE,3,../fasta/20231003/L11-3_G20.fasta,3958158,14,43295
131,L11-4_G20,11,4,20,20231003/L11-4_G20.fasta,r05,0,L11-4,L11-4_G20,20230909,FALSE,3,../fasta/20231003/L11-4_G20.fasta,4248613,29,92682


insert_id_ : insertion poses of query in ancestor \
new_ins_id : insertion pos in ref genome that is not in the ref \
lost_id: insertion pos in ancestor that is not in the query genome\
is_insert_id: insertion pos if true simple insertion or transposition \

note that lost here accounts for lost is-genome junctions and not the loss of is copies

In [25]:
def create_aggregate_data(is_in_r_df, suffix = ''):
	is_in_r_df_ = is_in_r_df.copy()
	## add corresponding ref pos_id for rows with original iss
	is_in_r_df_ = pd.merge(is_in_r_df_, is_in_r_df_.query('genome == "Ref"').loc[:, ['pos_id', 'insert_id', 'Line', 'Gen']].\
		rename(columns={'pos_id': 'pos_id_ref'}), on=['insert_id', 'Line', 'Gen'], how='left').\
			assign(pos_id_ref=lambda x: x.pos_id_ref.apply(lambda y: int(y) if not np.isnan(y) else -1))
	is_in_r_df_['insert_id_'] = is_in_r_df_.apply(lambda x: x.insert_id if x.genome == 'Query' else -1, axis=1)
	# for those with corresponding reference IS, assign the ID of the reference IS to overcount the number of insertions by counting both ends of the reference
	is_in_r_df_['insert_id_ref_check'] = is_in_r_df_.apply(lambda x: str(x.insert_id) if x.genome == 'Query' and x.position_status != 'original' else 'r' + str(x.pos_id_ref), axis=1)
	is_in_r_df_['new_ins_id'] = is_in_r_df_.apply(lambda x: x.insert_id if x.position_status == 'new' else -1, axis=1)
	is_in_r_df_['lost_id'] = is_in_r_df_.apply(lambda x: x.insert_id if x.position_status == 'lost' else -1, axis=1)
	is_in_r_df_['is_insertion'] = is_in_r_df_.apply(lambda x: x.insert_id if x.sv in ['simple_insertion', 'simple_transposition'] else -1, axis=1)

	aggregated_df = is_in_r_df_.groupby(['Line', 'Gen']).agg(
		ins_pos_cnt_raw=('insert_id', 'nunique'),
		ins_pos_cnt = ('insert_id_ref_check', 'nunique'),
		new_cnt = ('new_ins_id', 'nunique'),
		lost_end_cnt = ('lost_id', 'nunique'),
		simple_insertion_cnt=('is_insertion', 'nunique'),
	).eval('ins_pos_cnt_raw = ins_pos_cnt_raw - 1').\
		eval('ins_pos_cnt = ins_pos_cnt - 1').\
		eval('new_cnt = new_cnt - 1').\
		eval('lost_end_cnt = lost_end_cnt - 1').\
		eval('simple_insertion_cnt = simple_insertion_cnt - 1').\
		reset_index()
	# Get a list of all columns, excluding 'Line' and 'Gen'
	cols_to_change = [col for col in aggregated_df.columns if col not in ['Line', 'Gen']]
	rename_dict = {col: col + suffix for col in cols_to_change}
	# Rename the selected columns with suffixes
	aggregated_df.rename(columns=rename_dict, inplace=True)

	return aggregated_df

In [26]:
genome_stats = pd.merge(genome_stats, create_aggregate_data(is_in_r_df), how='left', left_on=['Prefix', 'gen_id'], right_on=['Line', 'Gen']).\
                     	drop(['Line', 'Gen'], axis=1).reset_index(drop=True)
genome_stats.to_csv(os.path.join(exportdir, 'classify_IS_events', 'genome_stats.csv'), index=False)
genome_stats.tail(5)

,IS_Detect_ID,ParentLine,SubLine,gen,file_name,Anc,RecA,Prefix,sample_name_raw,Contig_Date,...,gen_id,File,genome_size,is_cnt,is_length,ins_pos_cnt_raw,ins_pos_cnt,new_cnt,lost_end_cnt,simple_insertion_cnt
127,L10-4_G20,10,4,20,20231003/L10-4_G20.fasta,r04,0,L10-4,L10-4_G20,20230909,...,3,../fasta/20231003/L10-4_G20.fasta,4074199,25,99353,40.0,26.0,13.0,0.0,12.0
128,L11-1_G20,11,1,20,20231003/L11-1_G20.fasta,r05,0,L11-1,L11-1_G20,20230909,...,3,../fasta/20231003/L11-1_G20.fasta,4002703,20,61848,40.0,23.0,7.0,0.0,2.0
129,L11-2_G20,11,2,20,20231003/L11-2_G20.fasta,r05,0,L11-2,L11-2_G20,20230909,...,3,../fasta/20231003/L11-2_G20.fasta,4100257,25,77313,45.0,25.0,6.0,0.0,3.0
130,L11-3_G20,11,3,20,20231003/L11-3_G20.fasta,r05,0,L11-3,L11-3_G20,20230909,...,3,../fasta/20231003/L11-3_G20.fasta,3958158,14,43295,26.0,14.0,3.0,0.0,2.0
131,L11-4_G20,11,4,20,20231003/L11-4_G20.fasta,r05,0,L11-4,L11-4_G20,20230909,...,3,../fasta/20231003/L11-4_G20.fasta,4248613,29,92682,50.0,28.0,7.0,0.0,4.0


In [27]:
is_in_anc_df = analyse_stats.get_is_positions_in_ref_genome(lines_, os.path.join(exportdir, 'classify_IS_events_Anc'), [1, 2, 3])
print(is_in_anc_df.shape)
is_in_anc_df.to_csv(os.path.join(exportdir, 'classify_IS_events_Anc', 'IS_positions_in_ref_genome.csv'), index=False)
is_in_anc_df.tail(5)

(5779, 12)


,pos_id,IS_strand,flank,pos,genome,cluster_id,insert_id,flankMatchDir,position_status,sv,Line,Gen
5774,26,True,r,3756961,Query,6,40,False,new,unknown,L11-4,3
5775,27,True,f,3919604,Query,16,41,True,original,pristine,L11-4,3
5776,27,True,r,3922694,Query,13,42,False,original,pristine,L11-4,3
5777,28,True,f,3978541,Query,42,43,True,new,unknown,L11-4,3
5778,28,True,r,3980000,Query,21,44,False,new,unknown,L11-4,3


In [28]:
create_aggregate_data(is_in_anc_df, '_anc').tail(5)

,Line,Gen,ins_pos_cnt_raw_anc,ins_pos_cnt_anc,new_cnt_anc,lost_end_cnt_anc,simple_insertion_cnt_anc
83,L11-2,3,37,25,14,0,8
84,L11-3,2,26,14,3,0,0
85,L11-3,3,29,17,6,0,2
86,L11-4,2,36,24,13,0,8
87,L11-4,3,43,31,20,0,10


In [29]:
genome_stats = pd.merge(genome_stats,
                        create_aggregate_data(is_in_anc_df, '_anc'),
                        how='left', left_on=['Prefix', 'gen_id'], right_on=['Line', 'Gen']).\
                     	drop(['Line', 'Gen'], axis=1).reset_index(drop=True)
genome_stats.tail(5)

,IS_Detect_ID,ParentLine,SubLine,gen,file_name,Anc,RecA,Prefix,sample_name_raw,Contig_Date,...,ins_pos_cnt_raw,ins_pos_cnt,new_cnt,lost_end_cnt,simple_insertion_cnt,ins_pos_cnt_raw_anc,ins_pos_cnt_anc,new_cnt_anc,lost_end_cnt_anc,simple_insertion_cnt_anc
127,L10-4_G20,10,4,20,20231003/L10-4_G20.fasta,r04,0,L10-4,L10-4_G20,20230909,...,40.0,26.0,13.0,0.0,12.0,39.0,29.0,20.0,0.0,11.0
128,L11-1_G20,11,1,20,20231003/L11-1_G20.fasta,r05,0,L11-1,L11-1_G20,20230909,...,40.0,23.0,7.0,0.0,2.0,35.0,23.0,12.0,0.0,6.0
129,L11-2_G20,11,2,20,20231003/L11-2_G20.fasta,r05,0,L11-2,L11-2_G20,20230909,...,45.0,25.0,6.0,0.0,3.0,37.0,25.0,14.0,0.0,8.0
130,L11-3_G20,11,3,20,20231003/L11-3_G20.fasta,r05,0,L11-3,L11-3_G20,20230909,...,26.0,14.0,3.0,0.0,2.0,29.0,17.0,6.0,0.0,2.0
131,L11-4_G20,11,4,20,20231003/L11-4_G20.fasta,r05,0,L11-4,L11-4_G20,20230909,...,50.0,28.0,7.0,0.0,4.0,43.0,31.0,20.0,0.0,10.0


In [30]:
is_in_mds_df = analyse_stats.get_is_positions_in_ref_genome(lines_, os.path.join(exportdir, 'classify_IS_events_MDS'), [1, 2, 3])
print(is_in_mds_df.shape)
is_in_mds_df.to_csv(os.path.join(exportdir, 'classify_IS_events_MDS', 'IS_positions_in_ref_genome.csv'), index=False)
is_in_mds_df.tail(5)

(5000, 12)


,pos_id,IS_strand,flank,pos,genome,cluster_id,insert_id,flankMatchDir,position_status,sv,Line,Gen
4995,26,True,r,3727985,Query,17,32,False,new,unknown,L11-4,3
4996,27,True,f,3890676,Query,6,33,True,new,unknown,L11-4,3
4997,27,True,r,3898750,Query,26,34,False,new,unknown,L11-4,3
4998,28,True,f,3954602,Query,32,35,True,new,unknown,L11-4,3
4999,28,True,r,3956061,Query,18,36,False,new,unknown,L11-4,3


In [31]:
is_in_mds_df.head()

,pos_id,IS_strand,flank,pos,genome,cluster_id,insert_id,flankMatchDir,position_status,sv,Line,Gen
0,0,False,f,1272610,Ref,5,4,False,original,pristine,L01-1,1
1,0,False,r,1275701,Ref,6,5,True,original,pristine,L01-1,1
2,0,False,r,476936,Query,2,1,True,new,simple_insertion,L01-1,1
3,0,False,f,476923,Query,2,1,False,new,simple_insertion,L01-1,1
4,1,False,r,554076,Query,4,2,True,new,simple_insertion,L01-1,1


In [32]:
genome_stats = pd.merge(genome_stats,
                        create_aggregate_data(is_in_mds_df, '_mds'),
                        how='left', left_on=['Prefix', 'gen_id'], right_on=['Line', 'Gen']).\
                     	drop(['Line', 'Gen'], axis=1).reset_index(drop=True)
genome_stats.to_csv(os.path.join(exportdir, 'classify_IS_events', 'genome_stats_merged.csv'), index=False)
genome_stats.tail(5)

,IS_Detect_ID,ParentLine,SubLine,gen,file_name,Anc,RecA,Prefix,sample_name_raw,Contig_Date,...,ins_pos_cnt_raw_anc,ins_pos_cnt_anc,new_cnt_anc,lost_end_cnt_anc,simple_insertion_cnt_anc,ins_pos_cnt_raw_mds,ins_pos_cnt_mds,new_cnt_mds,lost_end_cnt_mds,simple_insertion_cnt_mds
127,L10-4_G20,10,4,20,20231003/L10-4_G20.fasta,r04,0,L10-4,L10-4_G20,20230909,...,39.0,29.0,20.0,0.0,11.0,30,29,29,0,15
128,L11-1_G20,11,1,20,20231003/L11-1_G20.fasta,r05,0,L11-1,L11-1_G20,20230909,...,35.0,23.0,12.0,0.0,6.0,26,25,25,0,13
129,L11-2_G20,11,2,20,20231003/L11-2_G20.fasta,r05,0,L11-2,L11-2_G20,20230909,...,37.0,25.0,14.0,0.0,8.0,28,27,27,0,13
130,L11-3_G20,11,3,20,20231003/L11-3_G20.fasta,r05,0,L11-3,L11-3_G20,20230909,...,29.0,17.0,6.0,0.0,2.0,20,19,19,0,7
131,L11-4_G20,11,4,20,20231003/L11-4_G20.fasta,r05,0,L11-4,L11-4_G20,20230909,...,43.0,31.0,20.0,0.0,10.0,35,34,34,0,16


# Aggregate sv data (MDS)

In [35]:
#C_ += ' --out ' + os.path.join(exportdir, 'classify_IS_events_MDS')

sv_df_mds = pd.DataFrame()
for line_ in lines:
	for qid in [1,2,3]:
		df_ = pd.read_csv(os.path.join(exportdir, 'classify_IS_events_MDS', line_, f'{line_}_genome.{qid}.sv_df.csv'))
		df_['Line'] = line_
		df_['gen_id'] = qid
		# inversion sizes should not exceed half the genome size
		genome_size_ = 3980425
		df_['SVLEN'] = df_.apply(lambda x: x.SVLEN if x.SVTYPE != 'INV' or x.SVLEN < genome_size_/2 else genome_size_ - x.SVLEN, axis=1)
		sv_df_mds = pd.concat([sv_df_mds, df_], axis=0)
sv_df_mds = sv_df_mds.reset_index(drop=True)
sv_df_mds.to_csv(os.path.join(exportdir, 'classify_IS_events_MDS', 'sv_df.csv'), index=False)
sv_df_mds.tail()

,chr1,start1,end1,REF,ALT,chr2,start2,end2,ID,parent,...,end2_is_status_r,new_cnt,dstart1_,dend1_,dstart2_,dend2_,SVLEN,SVLEN_fixed,Line,gen_id
2691,contig_1,3364619,3364619,t,tCGGTCAGTG,contig_1,3400998,3401007,INS651,SYN2,...,new,2,0,0,0.0,0.0,10,10,L11-4,3
2692,contig_1,3635997,3635997,t,tCACCACCGG,contig_1,3675419,3675428,INS713,SYN2,...,new,2,0,0,0.0,0.0,10,10,L11-4,3
2693,contig_1,3870530,3870612,ccgccttctgcgtcgctacaatggattcgattcccctcgggccaga...,c,contig_1,4142185,4142185,DEL772,SYN3,...,NaN,0,0,0,0.0,0.0,83,83,L11-4,3
2694,contig_1,3890673,3898751,-,-,contig_1,4162241,4162241,DEL4,SYN3,...,new,2,0,0,0.0,0.0,8079,8079,L11-4,3
2695,contig_1,3954599,3956062,-,-,contig_1,4221164,4221164,DEL5,SYN3,...,new,2,0,0,0.0,0.0,1464,1464,L11-4,3


In [36]:
sv_df_mds.query('SVTYPE == "HDR"').drop_duplicates(['start1', 'end1'], keep = 'first').\
    loc[:,['Line', 'gen_id', 'SVLEN_ISDEL', 'start1', 'end1', 'start2', 'end2', 'ID', 'start1_is_rm', 'end1_is_rm', 'start2_is_rm', 'end2_is_rm']]

,Line,gen_id,SVLEN_ISDEL,start1,end1,start2,end2,ID,start1_is_rm,end1_is_rm,start2_is_rm,end2_is_rm
261,L02-1,2,2398,3854237,3856634,3865103,3868203,HDR3,3851145,3853542,3805050,3805058
355,L02-2,3,49766,3670033,3719798,3408176,3440807,HDR4,3666941,3716706,3342640,3375271
772,L04-1,2,75492,2243177,2318668,1685689,1764283,HDR3,2240085,2315576,1657857,1733359
1114,L05-2,3,684004,1272609,1959704,2352785,2994379,HDR3,1272609,1956612,2289605,2911491
1125,L05-2,3,11366,3012354,3023719,1119735,1320078,HDR8,3009262,3020627,1093666,1287824
1515,L07-1,1,1157,2958682,2959838,2963669,2963822,HDR3,2955590,2956746,2932742,2932895
1695,L07-4,2,2282,846657,848938,849783,849790,HDR1,846657,848938,846689,846696
1899,L08-3,3,21308,1844426,1865733,1862503,1865595,HDR1,1841334,1862641,1840858,1840866
2442,L11-1,1,687,1786522,1787208,1798301,1799054,HDR1,1783430,1784116,1782835,1783588


In [37]:
genome_size_df

,Prefix,Gen,File,genome_size
0,L01-1,1,../fasta/20231004/L01_Anc_m1.fasta,4003851
1,L01-1,2,../fasta/20231013/L01-1_G8_m.fasta,4014776
2,L01-1,3,../fasta/20231003/L01-1_G20.fasta,3923637
3,L01-2,1,../fasta/20231004/L01_Anc_m1.fasta,4003851
4,L01-2,2,../fasta/20231003/L01-2_G8.fasta,4007666
...,...,...,...,...
127,L11-3,2,../fasta/20231003/L11-3_G8.fasta,3975273
128,L11-3,3,../fasta/20231003/L11-3_G20.fasta,3958158
129,L11-4,1,../fasta/20231003/L11_Anc.fasta,4004368
130,L11-4,2,../fasta/20231003/L11-4_G8.fasta,4007862


In [38]:
sv_df_evo = pd.DataFrame()
for line_ in lines:
	for qid in [2,3]:
		df_ = pd.read_csv(os.path.join(exportdir, 'classify_IS_events', line_, f'{line_}_genome.{qid}.sv_df.csv'))
		df_['Line'] = line_
		df_['gen_id'] = qid
		# genome size of reference genome
		genome_size_ = genome_size_df.query('Prefix == @line_ and Gen == @qid -1').genome_size.iloc[0]
		# inversion sizes should not exceed half the genome size
		df_['genome_size'] = genome_size_
		df_['SVLEN'] = df_.apply(lambda x: x.SVLEN if x.SVTYPE != 'INV' or x.SVLEN < genome_size_/2 else genome_size_ - x.SVLEN, axis=1)
		df_['SVLEN_fixed'] = df_.apply(lambda x: x.SVLEN_fixed if x.SVTYPE != 'INV' or x.SVLEN_fixed < genome_size_/2 else genome_size_ - x.SVLEN_fixed, axis=1)
		sv_df_evo = pd.concat([sv_df_evo, df_], axis=0)
sv_df_evo = sv_df_evo.reset_index(drop=True)
sv_df_evo.to_csv(os.path.join(exportdir, 'classify_IS_events', 'sv_df.csv'), index=False)
sv_df_evo.tail()

,chr1,start1,end1,REF,ALT,chr2,start2,end2,ID,parent,...,new_cnt,dstart1_,dend1_,dstart2_,dend2_,SVLEN,SVLEN_fixed,Line,gen_id,genome_size
989,contig_1,1873186,1873376,-,-,contig_1,1880592,1880592,DEL360,SYN1,...,1,0,1,0.0,0.0,191,192,L11-4,3,4007862
990,contig_1,2072930,2072930,G,GGGCATCGAC,contig_1,2080112,2080121,INS406,SYN1,...,2,0,0,0.0,0.0,10,10,L11-4,3,4007862
991,contig_1,2365287,2365287,T,TACGCGAAGA,contig_1,2375527,2375536,INS476,SYN1,...,2,0,0,0.0,0.0,10,10,L11-4,3,4007862
992,contig_1,167287,390321,-,-,contig_1,3773591,3996587,INVDP3,-,...,0,0,0,0.0,0.0,223035,223035,L11-4,3,4007862
993,contig_1,3978933,3980405,-,-,contig_1,4221164,4221164,DEL1,SYN2,...,1,0,0,0.0,0.0,1473,1473,L11-4,3,4007862


In [39]:
print("INV cnt %d" % len(sv_df_evo.query('SVTYPE == "INV"')))
print("max %d" % sv_df_evo.query('SVTYPE == "INV"').SVLEN_fixed.max())
print("min %d" % sv_df_evo.query('SVTYPE == "INV"').SVLEN_fixed.min())
print("mean %d" % sv_df_evo.query('SVTYPE == "INV"').SVLEN_fixed.mean())
print("median %d" % sv_df_evo.query('SVTYPE == "INV"').SVLEN_fixed.median())

INV cnt 55
max 1719098
min 2174
mean 243863
median 92322


In [40]:
sv_df_evo.query('SVTYPE == "INV"').sort_values('SVLEN_fixed', ascending=False).head(3)

,chr1,start1,end1,REF,ALT,chr2,start2,end2,ID,parent,...,new_cnt,dstart1_,dend1_,dstart2_,dend2_,SVLEN,SVLEN_fixed,Line,gen_id,genome_size
602,contig_1,883796,3146395,-,-,contig_1,888326,3122332,INV3,-,...,1,3093,0,0.0,-3094.0,1716005,1719098,L06-3,2,3978605
464,contig_1,884867,3399721,-,-,contig_1,891049,3397214,INV3,-,...,1,0,0,0.0,2.0,1419696,1419696,L05-2,3,3934551
531,contig_1,383001,3214409,-,-,contig_1,381924,3228692,INV5,-,...,1,0,-3092,0.0,0.0,1146661,1149753,L05-4,3,3978070


In [41]:
sv_df_evo.query('SVTYPE == "HDR"').drop_duplicates(['start1', 'end1'], keep = 'first')

,chr1,start1,end1,REF,ALT,chr2,start2,end2,ID,parent,...,new_cnt,dstart1_,dend1_,dstart2_,dend2_,SVLEN,SVLEN_fixed,Line,gen_id,genome_size
61,contig_1,1835423,1835600,-,-,contig_1,1836332,1848272,HDR2,SYN1,...,1,0,0,0.0,0.0,178,178,L01-3,3,4008366
103,contig_1,3894537,3900034,-,-,contig_1,3865103,3868203,HDR3,SYN1,...,1,0,-3098,0.0,0.0,5498,2400,L02-1,2,4026846
308,contig_1,2243314,2318818,-,-,contig_1,1685689,1761190,HDR2,INV3,...,0,0,1,-1.0,0.0,75505,75506,L04-1,2,3992834
400,contig_1,1859078,1944207,-,-,contig_1,1854139,1863203,HDR2,SYN1,...,1,0,2,0.0,0.0,85130,85132,L04-4,3,4025553
510,contig_1,2950687,2961668,-,-,contig_1,2862686,2865781,HDR8,SYN1,...,1,3093,0,0.0,0.0,10982,7889,L05-3,3,3869555
532,contig_1,473895,502346,-,-,contig_1,3127219,3137797,HDR1,INV5,...,2,0,0,-1.0,-3093.0,28452,28452,L05-4,3,3978070
538,contig_1,1701361,1722725,-,-,contig_1,1890157,2051835,HDR5,INV5,...,1,0,0,0.0,0.0,21365,21365,L05-4,3,3978070
539,contig_1,2004895,2008224,-,-,contig_1,1597401,1607980,HDR6,INV5,...,2,0,0,0.0,-3093.0,3330,3330,L05-4,3,3978070
684,contig_1,849738,852028,-,-,contig_1,849783,849790,HDR1,SYN1,...,1,0,0,0.0,0.0,2291,2291,L07-4,2,3998098
908,contig_1,1798301,1799054,-,-,contig_1,1807133,1807819,HDR1,SYN1,...,0,0,0,0.0,0.0,754,754,L11-1,2,4004368
